In [1]:
import three_desc_model as exponential_new
import numpy as np
import pandas as pd
from cutoffs import Polynomial
from scipy.optimize import minimize,fmin, differential_evolution
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import GPy
dataset_full = pd.read_csv('all_dataset.csv', index_col=0)
symbol="Siemens"
#dataset_pos=np.empty(shape=(0, 2))
#dataset_fp=np.empty(shape=(0, 3))
#dataset_ws=np.empty(shape=(0, 1))
k=0
count=0
dataset_full["IDnum"]=pd.Series()
for i in range(len(dataset_full)):
    dataset_full.at[i,"IDnum"]=k
    count = count +1
    if (count==dataset_full["Num_tot_turb"].iloc[i]):
        k=k+1
        count=0
dataset_full["Num_tot_turb"]=dataset_full["Num_tot_turb"].astype(int)
dataset_full["Turb_num"]=dataset_full["Turb_num"].astype(int)
dataset_full["IDnum"]=dataset_full["IDnum"].astype(int)
numsims=dataset_full["IDnum"].iloc[-1]+1
turb = [symbol]
nl=exponential_new.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)



In [2]:
def f(x):
    Gs = {"Siemens": [{"type":"G2", "turbine":"Siemens","eta":x[0], "offset": 1.},
                  {"type":"G4", "elements":"Siemens","eta":x[1], "gamma":4., "zeta": 1.0},
                  {"type":"G6", "elements":"Siemens","eta":x[2], "gamma": 0.5, "zeta": 1.0}
                 ]}
    finpr=exponential_new.FingerprintCalculator(cutoff=4001,Gs=Gs,Rct=3000,delta_R=100,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)
    count=0
    dataset_fp=np.empty(shape=(0, 3))
    dataset_ws=np.empty(shape=(0, 1))
    for i in range(numsims):
        numturb=dataset_full["Num_tot_turb"].iloc[count]
        position = np.empty((numturb,2))
        ws = np.empty((numturb,1))
        fp = np.empty((numturb,3))
        for k in range(numturb):
            position[k,0]=dataset_full.at[count,"X_coord"]
            position[k,1]=dataset_full.at[count,"Y_coord"]
            ws[k,0]=dataset_full.at[count,"Ref_wind"]
            count = count+ 1
        neigh=nl.calculate(turb*numturb,position)
        fingerprints=finpr.calculate(turb*numturb,position,neigh,symbol)
        fingerprints=np.array(fingerprints)
        #dataset_pos=np.append(dataset_pos,position,axis=0)
        dataset_fp=np.append(dataset_fp,fingerprints,axis=0)
        dataset_ws=np.append(dataset_ws,ws,axis=0)
    dataset=np.concatenate((dataset_fp, dataset_ws),axis=1)
    dataset = pd.DataFrame(dataset, columns = ['Fingerprint(G2)','Fingerprint(G4)','Fingerprint(G6)','Ref_Wind_Speed'])
    train_dataset = dataset#.sample(frac=0.8, random_state=0)
    test_dataset = dataset.drop(train_dataset.index)
    X=train_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)"]].to_numpy()
    Y=train_dataset[["Ref_Wind_Speed"]].to_numpy()
    Xnew=test_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)"]].to_numpy()
    Ytest=test_dataset[["Ref_Wind_Speed"]].to_numpy()
    # define kernel
    ker = GPy.kern.RBF(3,lengthscale=0.1)# + GPy.kern.White(2)
    # create simple GP model
    m = GPy.models.GPRegression(X,Y,ker)
    # optimize and plot
    m.optimize(max_f_eval = 1000)
    train_features = train_dataset.copy()
    #test_features = test_dataset.copy()
    all_features = dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)"]].to_numpy()
    #train_labels = train_features.pop('Ref_Wind_Speed')
    #test_labels = test_features.pop('Ref_Wind_Speed')
    all_labels = dataset.pop('Ref_Wind_Speed')


    [all_predictions,all_var] = m.predict(all_features)
    all_predictions
    error_all=np.abs(all_predictions[0]-all_labels)
    print(np.max(error_all))
    return np.max(error_all)

In [ ]:
i=np.linspace(4,6,2)

for k in i:
    x0 = [k, 5., 2.]
    print(x0)
    f(x0)


In [3]:
bnds = ((3, 6), (3, 6), (1,3))
x0 = [5.3, 5., 2.]
minimum = minimize(f, x0, bounds=bnds)

3.012161826871041
3.0121618270244044
3.012161826985637
3.012161827035148
3.0121624730495187
3.0121624730898207
3.0121624731134675
3.012162473124552
3.012161996482253
3.012161996430014
KeyboardInterrupt caught, calling on_optimization_end() to round things up


KeyboardInterrupt: 

In [ ]:
x0 = [5.3, 5., 2.]
bounds= ((0,10),(0,10),(0,10))
res = differential_evolution(f, x0, bounds= bounds, options={'disp':True})

In [ ]:
count=0
for i in range(numsims):
#    if dataset.loc[dataset['Number of Turbine in the Simulation'] == 2].reset_index(drop=True)
    numturb=dataset["Num_tot_turb"].iloc[count]
    print(count)
    position = np.empty((numturb,2))
    ws = np.empty((numturb,1))
    fp = np.empty((numturb,3))
    for k in range(numturb):
        position[k,0]=dataset.at[count,"X_coord"]
        position[k,1]=dataset.at[count,"Y_coord"]
        ws[k,0]=dataset.at[count,"Ref_wind"]
        count = count+ 1
    #print(position)
    neigh=nl.calculate(turb*numturb,position)
    fp=finpr.calculate(turb*numturb,position,neigh,symbol)
    fp=np.array(fp)
    print(fp)
    dataset_pos=np.append(dataset_pos,position,axis=0)
    dataset_fp=np.append(dataset_fp,fp,axis=0)
    dataset_ws=np.append(dataset_ws,ws,axis=0)
dataset=np.concatenate((dataset_fp, dataset_ws),axis=1)

In [ ]:
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

# Set-up hyperparameters
options = {'c1': 1.5, 'c2': 1.5, 'w':0.9}
# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=3, options=options, bounds=bounds)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=100)

In [ ]:
#--- MAIN ---------------------------------------------------------------------+

class Particle:
    def __init__(self,x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.9       # constant inertia weight (how much to weigh the previous velocity)
        c1=1       # cognative constant
        c2=0.1        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]
                
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print ('FINAL:')
        print (pos_best_g)
        print (err_best_g)

if __name__ == "__PSO__":
    main()

In [ ]:
#--- RUN ----------------------------------------------------------------------+
import random
x0 = [5.3, 5., 2.]
bounds= ((0,10),(0,10),(0,10))
PSO(f,x0,bounds,num_particles=10,maxiter=10)

#--- END ----------------------------------------------------------------------+